In [20]:
import pandas as pd
import nltk
import re
import gensim
import spacy
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
import pickle

df = pickle.load( open('processed_df','rb'))


In [21]:
lst_all = []
for i in list(set(df['EssaySet'])):
    lst_all.append(df[df['EssaySet'] == i])
# lst_all.append(df)

In [22]:
lst_train = []
lst_test = []
for dfrm in lst_all:
    dfrm = dfrm.sample(frac=1)
    eighty = int(0.8*dfrm.shape[0])
    lst_train.append(dfrm[:eighty])
    lst_test.append(dfrm[eighty:])

In [23]:

cvs = []
vectors_train = []
vectors_test = []
for i, dfrm in enumerate(lst_train):
    num_docs = lst_train[i].shape[0]
    min_df = int(num_docs/500)
    cvs.append(CountVectorizer(strip_accents='unicode', 
                                 ngram_range = (1,7),
                                 stop_words='english',
                                 min_df = min_df,
                                 max_df=0.9,
                                 lowercase=True))
    cvs[i].fit(dfrm['EssayPrepped'])
    vectors_train.append(cvs[i].transform(dfrm['EssayPrepped']))
    vectors_test.append(cvs[i].transform(lst_test[i]['EssayPrepped']))


In [24]:


groups_train = []
groups_test = []
ldas = []
kmns = []
for i, vector in enumerate(vectors_train):
    ldas.append(LatentDirichletAllocation(n_components=90, max_iter=3, n_jobs=1))
    kmns.append(KMeans(n_clusters=30))
    # append them
#     groups_train.append(np.append(vectors_train[i].todense(),ldas[i].fit_transform(vector), axis=1))
#     groups_test.append(np.append(vectors_test[i].todense(), ldas[i].transform(vectors_test[i]),axis=1))
    # ignore cvs after use
#     groups_train.append(ldas[i].fit_transform(vector))
#     groups_test.append(ldas[i].transform(vectors_test[i]))
    # use ONLY vectors
#     groups_train.append(vectors_train[i])
#     groups_test.append(vectors_test[i])
    # just argmax and the vector
#     groups_train.append(np.append(vectors_train[i].todense(), (ldas[i].fit_transform(vector) * np.max(vectors_test[i].todense())).astype(int), axis=1))
#     groups_test.append(np.append(vectors_test[i].todense(), (ldas[i].transform(vectors_test[i]) * np.max(vectors_test[i].todense())).astype(int),axis=1))
    # vector, lda(vector), kmeans(vecor)
    groups_train.append(np.append(
            np.append(
                vectors_train[i].todense(),
                ldas[i].fit_transform(vector), axis=1),
            kmns[i].fit_predict(vectors_train[i]).reshape(-1,1), axis=1))
    groups_test.append(np.append(
            np.append(
                vectors_test[i].todense(), 
                ldas[i].transform(vectors_test[i]),axis=1),
            kmns[i].predict(vectors_test[i]).reshape(-1,1), axis=1))
    
#     groups_train.append(np.append(vectors_train[i].todense(),
#                                   kmns[i].fit_transform(ldas[i].fit_transform(vector)), axis=1))
#     groups_test.append(np.append(vectors_test[i].todense(), 
#                                  kmns[i].transform(ldas[i].transform(vectors_test[i])),axis=1))
    # kmeans of lda of vector
#     groups_train.append(kmns[i].fit_transform(ldas[i].fit_transform(vector)))
#     groups_test.append(kmns[i].transform(ldas[i].transform(vectors_test[i])))
    
    # just asint...
#     groups_train.append( (ldas[i].fit_transform(vector) * 10).astype(int))
#     groups_test.append( (ldas[i].transform(vectors_test[i]) * 10).astype(int))
    # kmeans of vector
#     groups_train.append(kmns[i].fit_transform(vector))
#     groups_test.append(kmns[i].transform(vectors_test[i]))

C:\Users\Darin\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Users\Darin\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Users\Darin\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
C:\Users\Darin\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20.

In [25]:


proba1 = []
proba2 = []
models1 = []
models2 = []

for i, X in enumerate(groups_train):
    print(i,groups_train[i].shape,lst_train[i]['Score1'].shape)
    models1.append(GradientBoostingClassifier().fit(groups_train[i],lst_train[i]['Score1']))
    proba1.append(models1[i].predict(groups_test[i]))


0 (1337, 7500) (1337,)
1 (1022, 7812) (1022,)
2 (1512, 7446) (1512,)
3 (1390, 10921) (1390,)
4 (1436, 4069) (1436,)
5 (1437, 4277) (1437,)
6 (1439, 7263) (1439,)
7 (1439, 10439) (1439,)
8 (1438, 9126) (1438,)
9 (1312, 7745) (1312,)


In [26]:
import numpy as np

Score1 = []
Score2 = []

for i in range(len(proba1)):
    for a in proba1[i]:
        Score1.append(a)

df_test = pd.DataFrame(columns = lst_test[0].keys())
for dfrm in lst_test:
    df_test = df_test.append(dfrm)

df_test['Score1Pred'] = Score1


In [27]:
print('overall: {:.3f} vs {:.3f} [{:.3f}]'.format(
      df_test[df_test['Score1']==df_test['Score1Pred']].shape[0]/df_test.shape[0],
      df_test[df_test['Score1']==df_test['Score2']].shape[0]/df_test.shape[0],
       (df_test[df_test['Score1']==df_test['Score1Pred']].shape[0]/df_test.shape[0])/
       (df_test[df_test['Score1']==df_test['Score2']].shape[0]/df_test.shape[0])))
print()
for i in set(df_test['EssaySet']):
    df_test1 = df_test[df_test['EssaySet'] == i]
    print('Item {}: {:.3f} vs {:.3f} [{:.3f}]'.format(i,
          df_test1[df_test1['Score1']==df_test1['Score1Pred']].shape[0]/df_test1.shape[0],
          df_test1[df_test1['Score1']==df_test1['Score2']].shape[0]/df_test1.shape[0],
          df_test1[df_test1['Score1']==df_test1['Score1Pred']].shape[0]/df_test1[df_test1['Score1']==df_test1['Score2']].shape[0]))                                   



overall: 0.687 vs 0.873 [0.787]

Item 1: 0.561 vs 0.896 [0.627]
Item 2: 0.461 vs 0.891 [0.518]
Item 3: 0.528 vs 0.739 [0.714]
Item 4: 0.698 vs 0.784 [0.890]
Item 5: 0.875 vs 0.972 [0.900]
Item 6: 0.875 vs 0.967 [0.905]
Item 7: 0.719 vs 0.950 [0.757]
Item 8: 0.636 vs 0.819 [0.776]
Item 9: 0.706 vs 0.828 [0.852]
Item 10: 0.756 vs 0.896 [0.844]
